In [1]:
import pandas as pd
from haversine import haversine, Unit
from pyomo.environ import *
import plotly.express as px
import random

In [2]:
#!pip install pyomo haversine plotly


In [3]:
#!pip install glpk

In [4]:
# ================================================================
# 1 — DADOS DA TABELA (convertidos da imagem)
# ================================================================

df = pd.DataFrame({
    "provincia": [
        "Maputo Cidade", "Maputo Provincia", "Gaza", "Inhambane",
        "Zambézia", "Tete", "Manica", "Sofala",
        "Niassa", "Cabo Delgado", "Nampula"
    ],
    "lat": [-25.96, -25.70, -24.00, -23.86, -17.84, -16.19, -19.15, -19.83, -12.80, -12.33, -15.10],
    "lon": [32.57, 32.30, 33.75, 35.38, 36.89, 33.58, 33.47, 34.85, 35.00, 39.30, 39.25],
    "demanda": [659, 379, 497, 628, 1221, 668, 530, 524, 505, 558, 1350],
    "custo_fixo": [1435, 2459, 2733, 3247, 4818, 4310, 2976, 3058, 4273, 3923, 5275],
    "armazem_regional": ["Sul","Sul","Sul","Sul","Centro","Centro","Centro","Centro","Norte","Norte","Norte"]
})

# Armazéns regionais (Estimados)
R_coords = {
    "Sul": (-25.90, 32.60),
    "Centro": (-19.82, 34.84),
    "Norte": (-12.45, 39.00)
}

P = df["provincia"].tolist()
R = list(R_coords.keys())


In [5]:
df

,provincia,lat,lon,demanda,custo_fixo,armazem_regional
0,Maputo Cidade,-25.96,32.57,659,1435,Sul
1,Maputo Provincia,-25.70,32.30,379,2459,Sul
2,Gaza,-24.00,33.75,497,2733,Sul
3,Inhambane,-23.86,35.38,628,3247,Sul
4,Zambézia,-17.84,36.89,1221,4818,Centro
5,Tete,-16.19,33.58,668,4310,Centro
6,Manica,-19.15,33.47,530,2976,Centro
7,Sofala,-19.83,34.85,524,3058,Centro
8,Niassa,-12.80,35.00,505,4273,Norte
9,Cabo Delgado,-12.33,39.30,558,3923,Norte


In [6]:
R_coords

{'Sul': (-25.9, 32.6), 'Centro': (-19.82, 34.84), 'Norte': (-12.45, 39.0)}

In [7]:
P

['Maputo Cidade',
 'Maputo Provincia',
 'Gaza',
 'Inhambane',
 'Zambézia',
 'Tete',
 'Manica',
 'Sofala',
 'Niassa',
 'Cabo Delgado',
 'Nampula']

In [8]:
R

['Sul', 'Centro', 'Norte']

In [9]:
# Dicionários
D = dict(zip(df["provincia"], df["demanda"]))
Fix = dict(zip(df["provincia"], df["custo_fixo"]))
Region = dict(zip(df["provincia"], df["armazem_regional"]))
coords = dict(zip(df["provincia"], zip(df["lat"], df["lon"])))

In [10]:
D

{'Maputo Cidade': 659,
 'Maputo Provincia': 379,
 'Gaza': 497,
 'Inhambane': 628,
 'Zambézia': 1221,
 'Tete': 668,
 'Manica': 530,
 'Sofala': 524,
 'Niassa': 505,
 'Cabo Delgado': 558,
 'Nampula': 1350}

In [11]:
Fix

{'Maputo Cidade': 1435,
 'Maputo Provincia': 2459,
 'Gaza': 2733,
 'Inhambane': 3247,
 'Zambézia': 4818,
 'Tete': 4310,
 'Manica': 2976,
 'Sofala': 3058,
 'Niassa': 4273,
 'Cabo Delgado': 3923,
 'Nampula': 5275}

In [12]:
Region

{'Maputo Cidade': 'Sul',
 'Maputo Provincia': 'Sul',
 'Gaza': 'Sul',
 'Inhambane': 'Sul',
 'Zambézia': 'Centro',
 'Tete': 'Centro',
 'Manica': 'Centro',
 'Sofala': 'Centro',
 'Niassa': 'Norte',
 'Cabo Delgado': 'Norte',
 'Nampula': 'Norte'}

In [13]:
coords

{'Maputo Cidade': (-25.96, 32.57),
 'Maputo Provincia': (-25.7, 32.3),
 'Gaza': (-24.0, 33.75),
 'Inhambane': (-23.86, 35.38),
 'Zambézia': (-17.84, 36.89),
 'Tete': (-16.19, 33.58),
 'Manica': (-19.15, 33.47),
 'Sofala': (-19.83, 34.85),
 'Niassa': (-12.8, 35.0),
 'Cabo Delgado': (-12.33, 39.3),
 'Nampula': (-15.1, 39.25)}

In [14]:
# ================================================================
# 2 — DISTÂNCIAS E CUSTOS POR KM
# ================================================================

CUSTO_POR_KM = 1.25

DIST = {(p,r): haversine(coords[p], R_coords[r], unit=Unit.KILOMETERS)
        for p in P for r in R}

Ctr = {(p,r): DIST[(p,r)] * CUSTO_POR_KM for p in P for r in R}


In [15]:
DIST

{('Maputo Cidade', 'Sul'): 7.315177063594022,
 ('Maputo Cidade', 'Centro'): 721.195830860178,
 ('Maputo Cidade', 'Norte'): 1646.1133232275276,
 ('Maputo Provincia', 'Sul'): 37.37072811731548,
 ('Maputo Provincia', 'Centro'): 703.7242660143917,
 ('Maputo Provincia', 'Norte'): 1631.9936774618413,
 ('Gaza', 'Sul'): 240.98839347873155,
 ('Gaza', 'Centro'): 478.1956023811658,
 ('Gaza', 'Norte'): 1398.3884698548216,
 ('Inhambane', 'Sul'): 360.66842072286585,
 ('Inhambane', 'Centro'): 452.6703673736658,
 ('Inhambane', 'Norte'): 1324.8932546105625,
 ('Zambézia', 'Sul'): 999.3595904667305,
 ('Zambézia', 'Centro'): 308.2442724294732,
 ('Zambézia', 'Norte'): 640.6664807847208,
 ('Tete', 'Sul'): 1084.4670723643092,
 ('Tete', 'Centro'): 425.05296619018713,
 ('Tete', 'Norte'): 716.7892934819856,
 ('Manica', 'Sul'): 755.8582080500385,
 ('Manica', 'Centro'): 161.78547020393944,
 ('Manica', 'Norte'): 951.105209565863,
 ('Sofala', 'Sul'): 713.1828614507521,
 ('Sofala', 'Centro'): 1.5266474439974724,
 ('

In [16]:
Ctr

{('Maputo Cidade', 'Sul'): 9.143971329492528,
 ('Maputo Cidade', 'Centro'): 901.4947885752225,
 ('Maputo Cidade', 'Norte'): 2057.6416540344094,
 ('Maputo Provincia', 'Sul'): 46.71341014664435,
 ('Maputo Provincia', 'Centro'): 879.6553325179896,
 ('Maputo Provincia', 'Norte'): 2039.9920968273018,
 ('Gaza', 'Sul'): 301.2354918484144,
 ('Gaza', 'Centro'): 597.7445029764573,
 ('Gaza', 'Norte'): 1747.985587318527,
 ('Inhambane', 'Sul'): 450.8355259035823,
 ('Inhambane', 'Centro'): 565.8379592170822,
 ('Inhambane', 'Norte'): 1656.1165682632031,
 ('Zambézia', 'Sul'): 1249.1994880834131,
 ('Zambézia', 'Centro'): 385.3053405368415,
 ('Zambézia', 'Norte'): 800.833100980901,
 ('Tete', 'Sul'): 1355.5838404553865,
 ('Tete', 'Centro'): 531.316207737734,
 ('Tete', 'Norte'): 895.986616852482,
 ('Manica', 'Sul'): 944.822760062548,
 ('Manica', 'Centro'): 202.2318377549243,
 ('Manica', 'Norte'): 1188.8815119573287,
 ('Sofala', 'Sul'): 891.4785768134401,
 ('Sofala', 'Centro'): 1.9083093049968405,
 ('Sofal

In [17]:
# ================================================================
# 3 — VRP SIMPLIFICADO (Nearest Neighbor)
# ================================================================

def vrp_cost(prov):
    random.seed(1)
    n = 6  # unidades sanitárias simuladas
    us_points = [
        (coords[prov][0] + random.uniform(-0.05, 0.05),
         coords[prov][1] + random.uniform(-0.05, 0.05))
        for _ in range(n)
    ]

    points = [coords[prov]] + us_points
    m = len(points)

    dist = [[haversine(points[i], points[j], unit=Unit.KILOMETERS)
             for j in range(m)] for i in range(m)]

    visited = {0}
    route = [0]
    current = 0
    total = 0

    while len(visited) < m:
        nxt = min(
            [j for j in range(m) if j not in visited],
            key=lambda j: dist[current][j]
        )
        total += dist[current][nxt]
        visited.add(nxt)
        current = nxt

    total += dist[current][0]

    return total * CUSTO_POR_KM

In [18]:
VRP_costs = {p: vrp_cost(p) for p in P}


In [19]:
VRP_costs

{'Maputo Cidade': 38.29060589247991,
 'Maputo Provincia': 38.33986946582133,
 'Gaza': 38.65102824942596,
 'Inhambane': 38.67579829069881,
 'Zambézia': 39.613760856516855,
 'Tete': 39.82621177597204,
 'Manica': 39.431197661580164,
 'Sofala': 39.331639571059775,
 'Niassa': 40.200409304103324,
 'Cabo Delgado': 40.2455849241736,
 'Nampula': 39.95573395751712}

In [20]:
# ================================================================
# 4 — MODELO DE OTIMIZAÇÃO (Pyomo)
# ================================================================

model = ConcreteModel()

model.P = Set(initialize=P)
model.R = Set(initialize=R)

# Variáveis
model.x_NR = Var(model.R, domain=NonNegativeReals)
model.x_RP = Var(model.R, model.P, domain=NonNegativeReals)

# Objetivo
def obj_rule(m):
    transporte = sum(Ctr[p,r] * m.x_RP[r,p] for p in m.P for r in m.R)
    fixos = sum(Fix[p] for p in m.P)
    vrp = sum(VRP_costs[p] for p in m.P)
    return transporte + fixos + vrp

model.obj = Objective(rule=obj_rule, sense=minimize)

# Demanda
def demanda_rule(m, p):
    return sum(m.x_RP[r,p] for r in m.R) == D[p]
model.demanda = Constraint(model.P, rule=demanda_rule)

# Compatibilidade geográfica
def geo_rule(m, r, p):
    return m.x_RP[r,p] <= (D[p] if Region[p] == r else 0)
model.geo = Constraint(model.R, model.P, rule=geo_rule)

# Nacional → Regional
def fluxo_rule(m, r):
    return m.x_NR[r] >= sum(D[p] for p in m.P if Region[p] == r)
model.fluxo = Constraint(model.R, rule=fluxo_rule)


In [21]:

# ================================================================
# 5 — Resolver
# ================================================================

solver = SolverFactory("glpk")
results = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp3z3vtvbn.glpk.raw --wglp /tmp/tmppv3ovprb.glpk.glp --cpxlp
 /tmp/tmpo8ueicla.pyomo.lp
Reading problem data from '/tmp/tmpo8ueicla.pyomo.lp'...
47 rows, 37 columns, 69 non-zeros
290 lines were read
Writing problem data to '/tmp/tmppv3ovprb.glpk.glp'...
238 lines were written
GLPK Simplex Optimizer 5.0
47 rows, 37 columns, 69 non-zeros
Preprocessing...
~     0: obj =   2.228034195e+06  infeas =  0.000e+00
OPTIMAL SOLUTION FOUND BY LP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.0 Mb (48420 bytes)
Writing basic solution to '/tmp/tmp3z3vtvbn.glpk.raw'...
93 lines were written


In [22]:
#print("\n======== RESULTADOS ========")
print("CUSTO TOTAL DO SISTEMA = ", model.obj())

CUSTO TOTAL DO SISTEMA =  2228034.195398081


In [23]:
#print("\nCustos VRP (rotas internas):")
for p in P:
    print(f"- {p}: {VRP_costs[p]:.2f} USD")


- Maputo Cidade: 38.29 USD
- Maputo Provincia: 38.34 USD
- Gaza: 38.65 USD
- Inhambane: 38.68 USD
- Zambézia: 39.61 USD
- Tete: 39.83 USD
- Manica: 39.43 USD
- Sofala: 39.33 USD
- Niassa: 40.20 USD
- Cabo Delgado: 40.25 USD
- Nampula: 39.96 USD


In [24]:
# ================================================================
# 6 — DASHBOARD (PLOTLY)
# ================================================================

df_map = df.copy()
df_map["Tipo"] = "Província"

# adicionar armazéns regionais
df_ar = pd.DataFrame({
    "provincia": R,
    "lat": [R_coords[r][0] for r in R],
    "lon": [R_coords[r][1] for r in R],
    "Tipo": "Armazém Regional"
})

df_map = pd.concat([df_map, df_ar], ignore_index=True)

In [25]:
fig = px.scatter_geo(
    df_map,
    lat="lat",
    lon="lon",
    color="Tipo",
    hover_name="provincia",
    title="Mapa Logístico — SNS Moçambique",
    projection="natural earth"
)
fig.update_layout(height=600)
fig.show()


/home/jossian/.local/lib/python3.10/site-packages/plotly/express/_core.py:1980: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])
